<a href="https://colab.research.google.com/github/garfield-gray/NeuralNetwork/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import sklearn as sl
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sb


In [ ]:
tf.keras.layers.Dropout(rate = 0.3)